In [17]:
import pandas as pd
import numpy as np

def table_results_to_excel(Country,first_year,last_year):
    
    if Country == 'Portugal':
        country = 'portugal'
    elif Country == 'Spain':
        country = 'primera'
    elif Country == 'England':
        country = 'premier'
    elif Country == 'Italy':
        country = 'serie_a'
    elif Country == 'Germany':
        country = 'bundesliga'
    elif Country == 'France':
        country = 'ligue_1'
    else:
        print('Country not available, please choose between Portugal,\
 Spain, England, Italy, Germany or France')
    
    
    first_year = first_year
    last_year = last_year

    number_of_teams_per_year = []
    weekdays_total_per_year = []

    for year_ in range(first_year,last_year+1):
        URL = f'https://www.resultados-futbol.com/{country}{year_}/grupo1/equipos'
        webpage_equipos = pd.read_html(URL)
        Table_year = pd.DataFrame(webpage_equipos[len(webpage_equipos)-1])
        number_of_teams_per_year.append(len(Table_year['Equipo.1']))

    for number_of_teams_ in number_of_teams_per_year:
        weekdays_total_per_year.append((number_of_teams_-1)*2)


    list_of_years_times_weekdays = []
    list_of_weekdays_times_years = []

    list_of_years = list(range(first_year,last_year+1))

    for i in range(len(list_of_years)):
        for weekday_ in range(weekdays_total_per_year[i]):
            for number_of_games_per_weekday_ in range(int(number_of_teams_per_year[i]/2)):
                list_of_years_times_weekdays.append(list_of_years[i])

    for i in range(len(list_of_years)):
        for weekday_ in range(1,weekdays_total_per_year[i]+1):
            for number_of_games_per_weekday_ in range(int(number_of_teams_per_year[i]/2)):
                list_of_weekdays_times_years.append(weekday_)



    #home_teams_year_weekday, away_teams_year_weekday, only_results_year_weekday,      

    date_list = []
    home_team_list = []
    away_team_list = []
    results_list = []

    for i in range(len(list_of_years)):
        for m in range(1,weekdays_total_per_year[i]+1):
            URL = f'https://www.resultados-futbol.com/{country}{list_of_years[i]}/grupo1/jornada{m}'

            webpage = pd.read_html(URL)
            weekday_table = pd.DataFrame(webpage[7])
            number_columns = range(weekday_table.shape[1])
            weekday_table.columns = ['To delete', 'Date', 'Home_team',
                                       'Description', 'Away_team', 'More_info']
            weekday_table = weekday_table.drop(['To delete'], axis=1)
            weekday_table = weekday_table.dropna()

            list_final_results_weekday = []

            for m in list(weekday_table['Description']):
                if " " not in m[m.rfind('-'):]:
                    list_final_results_weekday.append(m[m.rindex(' ')+1:])
                else:
                    list_final_results_weekday.append(m[m.rindex(' ',0,m.rindex(' '))+1:m.rindex(' ')])

            weekday_table.insert(3, 'Final_Result', list_final_results_weekday, True)    

            for full_date in list(weekday_table['Date']):
                date_list.append(str(full_date)[0:9])

            home_team_list.append(list(weekday_table['Home_team']))
            away_team_list.append(list(weekday_table['Away_team']))
            results_list.append((weekday_table['Final_Result']))


    # each one of these lists is a list of lists, and we need only one list

    date_list_only_one_list = []
    for sublist in date_list:
        for item in sublist:
            date_list_only_one_list.append(item)

    home_team_list_only_one_list = []
    for sublist in home_team_list:
        for item in sublist:
            home_team_list_only_one_list.append(item)

    away_team_list_only_one_list = []
    for sublist in away_team_list:
        for item in sublist:
            away_team_list_only_one_list.append(item)

    results_list_only_one_list = []
    for sublist in results_list:
        for item in sublist:
            results_list_only_one_list.append(item)


    #DataFrame definition

    dfdata_table_results = {
              'Country': Country,
              'Year': list_of_years_times_weekdays,
              'Date': date_list,
              'Weekday': list_of_weekdays_times_years,
              'Home_team': home_team_list_only_one_list,
              'Away_team': away_team_list_only_one_list,
              'Result': results_list_only_one_list
             }

    Table_Results_web_scrapping = pd.DataFrame(dfdata_table_results)

    #DataFrame enrichment

    list_all_scores = list(Table_Results_web_scrapping['Result'])
    list_home_score = []
    list_away_score = []

    for score in list_all_scores:
        list_home_score.append(score[:score.index('-')])
        list_away_score.append(score[score.index('-')+1:])

    Table_Results_web_scrapping['Home_score'] = list_home_score
    Table_Results_web_scrapping['Away_score'] = list_away_score
    Table_Results_web_scrapping.loc[Table_Results_web_scrapping['Home_score'] == Table_Results_web_scrapping['Away_score'], '1x2'] = 'x'
    Table_Results_web_scrapping.loc[Table_Results_web_scrapping['Home_score'] > Table_Results_web_scrapping['Away_score'], '1x2'] = '1'
    Table_Results_web_scrapping.loc[Table_Results_web_scrapping['Home_score'] < Table_Results_web_scrapping['Away_score'], '1x2'] = '2'
    Table_Results_web_scrapping.loc[Table_Results_web_scrapping['1x2'] == 'x', 'Points_Home_Team'] = int('1')
    Table_Results_web_scrapping.loc[Table_Results_web_scrapping['1x2'] == '1', 'Points_Home_Team'] = int('3')
    Table_Results_web_scrapping.loc[Table_Results_web_scrapping['1x2'] == '2', 'Points_Home_Team'] = int('0')
    Table_Results_web_scrapping.loc[Table_Results_web_scrapping['1x2'] == 'x', 'Points_Away_Team'] = int('1')
    Table_Results_web_scrapping.loc[Table_Results_web_scrapping['1x2'] == '1', 'Points_Away_Team'] = int('0')
    Table_Results_web_scrapping.loc[Table_Results_web_scrapping['1x2'] == '2', 'Points_Away_Team'] = int('3')
    
    Table_Results_web_scrapping.to_excel(f'Table_results_{Country}_{first_year}_{last_year}.xlsx')

In [18]:
table_results_to_excel('England',2012,2012)

In [19]:
Country = 'England'
first_year = 2012
last_year = 2012

Table_Results = pd.read_excel(f'Table_results_{Country}_{first_year}_{last_year}.xlsx')
Table_Results.drop(['Unnamed: 0'],axis = 1, inplace = True)
Table_Results

,Country,Year,Date,Weekday,Home_team,Away_team,Result,Home_score,Away_score,1x2,Points_Home_Team,Points_Away_Team
0,England,2012,13 Ago 11,1,Blackburn Rovers,Wolves,1-2,1,2,2,0,3
1,England,2012,13 Ago 11,1,Fulham,Aston Villa,0-0,0,0,x,1,1
2,England,2012,13 Ago 11,1,Liverpool,Sunderland,1-1,1,1,x,1,1
3,England,2012,13 Ago 11,1,Queens Park Rangers,Bolton Wanderers,0-4,0,4,2,0,3
4,England,2012,13 Ago 11,1,Wigan Athletic,Norwich City,1-1,1,1,x,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
375,England,2012,13 May 12,38,Sunderland,Man. Utd,0-1,0,1,2,0,3
376,England,2012,13 May 12,38,Swansea City,Liverpool,1-0,1,0,1,3,0
377,England,2012,13 May 12,38,Tottenham Hotspur,Fulham,2-0,2,0,1,3,0
378,England,2012,13 May 12,38,West Bromwich Albion,Arsenal,2-3,2,3,2,0,3
